#### 9. mongodb에 크롤링 데이터 저장하기

In [1]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

actors_info_list = list()

cine21_url = 'http://www.cine21.com/rank/person/content'
post_data = dict()
post_data['section']='actor'
post_data['period_start']='2020-12'
post_data['gender']='all'

for index in range(1,21):
    post_data['page'] = index
    
    res = requests.post(cine21_url, data=post_data)
    soup = BeautifulSoup(res.content, 'html.parser')
    
    actors = soup.select('li.people_li div.name > a') # 배우이름
    hits = soup.select('ul.num_info > li > strong') # 흥행지수
    movies = soup.select('ul.mov_list') # 출연영화 
    rankings = soup.select('li.people_li > span.grade') # 랭킹

    for index, actor in enumerate(actors):
        actor_name = re.sub('\(\w*\)','',actor.text)
        actor_hits = int(hits[index].text.replace(',',''))
        movie_titles = movies[index].select('li a span')
        movie_title_list = list()
        for movie_title in movie_titles:
            movie_title_list.append(movie_title.text)

        actor_info_dict = dict()
        actor_info_dict['배우이름'] = actor_name
        actor_info_dict['흥행지수'] = actor_hits
        actor_info_dict['출연영화'] = movie_title_list
        actor_info_dict['랭킹'] = rankings[index].text

        actor_link = 'http://www.cine21.com' + actor.attrs['href'] # 배우 링크
        response_actor = requests.get(actor_link) # 배우 링크 가져오기
        soup_actor = BeautifulSoup(response_actor.content, 'html.parser') # 배우 링크 파싱하기
        default_info = soup_actor.select_one('ul.default_info') # 배우 기본 정보
        actor_details = default_info.select('li') # 기본 정보내에서 상세정보 데이터 추출

        for actor_item in actor_details:
            actor_item_field = actor_item.select_one('span.tit').text # 원하는 값
            actor_item_value = re.sub('<span.*?>.*?</span>', '', str(actor_item))
            actor_item_value = re.sub('<.*?>', '', actor_item_value) # 원하는 값
            actor_info_dict[actor_item_field] = actor_item_value # 딕셔너리로 저장 'field':'value'
        actors_info_list.append(actor_info_dict)
        
actor_collection.insert_many(actors_info_list)

In [4]:
conn = pymongo.MongoClient()
actor_db = conn.cine21
actor_collection = actor_db.actor_collection

actor_collection.insert_many(actors_info_list)

In [3]:
actor_collection.delete_many({})